# Step 2: Agents in a Workflow (Non-Streaming)

## Overview

This notebook demonstrates how to integrate AI agents into workflows using the Agent Framework. We'll create a two-agent workflow where:
1. **Writer Agent** - Creates or edits content
2. **Reviewer Agent** - Evaluates and provides feedback

### Key Concepts:

- **Agents as Workflow Executors**: Wrap chat agents created by `AzureOpenAIChatClient` inside workflow executors
- **Automatic Output Yielding**: Agents automatically yield outputs when they complete
- **Non-Streaming Execution**: Using `workflow.run()` for synchronous results
- **Agent Events**: Capturing and displaying agent run events

### Prerequisites:

- ✅ Azure OpenAI configured with required environment variables
- ✅ Azure CLI authentication (`az login` completed)
- ✅ Basic familiarity with WorkflowBuilder, executors, and edges

## Import Required Libraries

In [ ]:
import asyncio

from agent_framework import AgentRunEvent, WorkflowBuilder
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

## Create Azure OpenAI Chat Client

We'll use **Azure OpenAI** with `AzureCliCredential` for authentication.

### Environment Variables Required:
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI endpoint URL
- **AZURE_OPENAI_CHAT_DEPLOYMENT_NAME**: Your model deployment name (e.g., "gpt-4o")

These should be set in the `.env` file located at `python/samples/getting_started/.env`

### Authentication:
`AzureCliCredential` uses Azure CLI authentication (`az login`).

In [ ]:
# Verify environment variables are loaded
print("🔍 Checking Azure OpenAI environment variables...")
print(f"AZURE_OPENAI_ENDPOINT: {os.getenv('AZURE_OPENAI_ENDPOINT', 'Not set')}")
print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'Not set')}")

In [ ]:
# Create the Azure OpenAI chat client
# Get the endpoint and deployment name from environment variables
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if not endpoint:
    raise ValueError(
        "❌ Azure OpenAI endpoint not found. "
        "Please set AZURE_OPENAI_ENDPOINT in your .env file"
    )

if not deployment_name:
    raise ValueError(
        "❌ Azure OpenAI deployment name not found. "
        "Please set AZURE_OPENAI_CHAT_DEPLOYMENT_NAME in your .env file"
    )

print(f"🔧 Using Azure OpenAI Endpoint: {endpoint}")
print(f"🔧 Using Deployment: {deployment_name}")

# Create the Azure OpenAI chat client
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)

print("✅ Azure OpenAI Chat Client created successfully!")

## Define the Writer Agent

The Writer Agent is responsible for creating and editing content based on requirements and feedback.

**Agent Configuration:**
- **Instructions**: Guides the agent's behavior and role
- **Name**: Identifier for the agent (used in events and debugging)

In [ ]:
writer_agent = chat_client.create_agent(
    instructions=(
        "You are an excellent content writer. You create new content and edit contents based on the feedback."
    ),
    name="writer",
)

print("✅ Writer Agent created")

## Define the Reviewer Agent

The Reviewer Agent evaluates content and provides actionable feedback.

**Key Characteristics:**
- Provides concise, actionable feedback
- Evaluates content quality
- Finalizes the result

In [ ]:
reviewer_agent = chat_client.create_agent(
    instructions=(
        "You are an excellent content reviewer."
        "Provide actionable feedback to the writer about the provided content."
        "Provide the feedback in the most concise manner possible."
    ),
    name="reviewer",
)

print("✅ Reviewer Agent created")

## Build the Workflow

### Workflow Structure:

```
Writer Agent → Reviewer Agent
```

Using the fluent `WorkflowBuilder` API:
1. Set the writer as the start node
2. Connect an edge from writer to reviewer
3. Build the workflow

In [ ]:
# Build the workflow using the fluent builder
workflow = (
    WorkflowBuilder()
    .set_start_executor(writer_agent)
    .add_edge(writer_agent, reviewer_agent)
    .build()
)

print("✅ Workflow built successfully!")
print("   Writer → Reviewer")

## Run the Workflow

### Execution Flow:

1. User provides initial message
2. Writer Agent creates content based on the request
3. Writer's output is automatically sent to Reviewer Agent
4. Reviewer Agent evaluates and provides feedback
5. Workflow completes when all agents are done

### Event Handling:

- `AgentRunEvent` - Captures agent responses
- `get_outputs()` - Retrieves final workflow outputs
- `get_final_state()` - Shows workflow completion status

In [ ]:
# Run the workflow with a user message
user_message = "Create a slogan for a new electric SUV that is affordable and fun to drive."

print(f"\n📝 User Request: {user_message}\n")
print("=" * 60)

# Execute the workflow
events = await workflow.run(user_message)

# Print agent run events
print("\n🤖 Agent Responses:\n")
for event in events:
    if isinstance(event, AgentRunEvent):
        print(f"{event.executor_id}: {event.data}")

# Print final outputs
print(f"\n{'=' * 60}")
print("\n📤 Workflow Outputs:")
for output in events.get_outputs():
    print(output)

# Print final state
print(f"\n✅ Final state: {events.get_final_state()}")

## Expected Output

### Sample Execution:

```
writer: "Charge Up Your Adventure—Affordable Fun, Electrified!"

reviewer: Slogan: "Plug Into Fun—Affordable Adventure, Electrified."

**Feedback:**
- Clear focus on affordability and enjoyment.
- "Plug into fun" connects emotionally and highlights electric nature.
- Consider specifying "SUV" for clarity in some uses.
- Strong, upbeat tone suitable for marketing.

============================================================
Workflow Outputs: ['Slogan: "Plug Into Fun—Affordable Adventure, Electrified."

**Feedback:**
- Clear focus on affordability and enjoyment.
- "Plug into fun" connects emotionally and highlights electric nature.
- Consider specifying "SUV" for clarity in some uses.
- Strong, upbeat tone suitable for marketing.']

Final state: WorkflowRunState.COMPLETED
```

## Key Takeaways

### Agent Integration

✅ **Agents as Executors**
- AI agents created with `create_agent()` can be used directly in workflows
- No need for custom executor wrappers
- Seamless integration with WorkflowBuilder

✅ **Automatic Output Handling**
- Agents automatically yield outputs when they complete
- No explicit `ctx.yield_output()` needed
- Simplifies workflow construction

✅ **Event-Driven Architecture**
- `AgentRunEvent` captures agent responses
- Events can be filtered and processed
- Full visibility into workflow execution

### Non-Streaming vs Streaming

**Non-Streaming (`workflow.run()`):**
- Waits for complete responses
- Returns all events at once
- Simpler for batch processing
- Used in this example

**Streaming (`workflow.run_stream()`):**
- Real-time response chunks
- Better user experience for long responses
- Covered in Step 3

### Workflow Pattern

This example demonstrates a **Sequential Agent Chain**:
```
User Input → Writer Agent → Reviewer Agent → Final Output
```

Common use cases:
- Content creation and review
- Multi-stage processing pipelines
- Quality assurance workflows

### Next Steps

Continue to **Step 3** to learn about streaming responses and real-time output!